# MIMIC-III Multi-Modal Data Creation

# Clone the repository
!git clone https://github.com/YerevaNN/mimic3-benchmarks

# Follow the instructions in the README file:
# 1. Requirements and installation
# 2. Build benchmark: Use only the first command
!python -m mimic3benchmark.scripts.extract_subjects {PATH_TO_MIMIC_III_CSVs} data/root/

# The output will include a CSV file in the `data` directory, which you need to use as input for further processing.



In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import getpass

import sns as sns
%matplotlib inline
plt.style.use('ggplot') 

In [2]:
# Create a database connection
user = 'postgres'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'

In [3]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))
query_schema = 'set search_path to ' + schema+ ';'

In [65]:
#read the table from path
path = 'enter the path of the output_file'
data = pd.read_csv(path)
data.columns = map(str.lower, data.columns)



In [66]:
# Load labevents data (this table might be large, so filter only necessary rows)
labevents_query = query_schema+"""
SELECT subject_id, itemid
FROM labevents
WHERE itemid IN (50813, 50820, 51491, 50831, 51094, 50818, 50809, 51478, 50931,
                 50902, 50821, 50833, 50971, 50862, 50882, 50885, 50802, 50983,
                 50868, 50825, 51222, 50811, 51221, 51480, 220045, 211, 220210, 618) AND labevents.FLAG = 'abnormal'
"""
labevents_df = pd.read_sql_query(labevents_query, con)
labevents_pivot = labevents_df.assign(flag=1).pivot_table(
    index='subject_id',
    columns='itemid',
    values='flag',
    fill_value=0
).reset_index()
# Map itemid to descriptive names
itemid_to_name = {
    50813: 'Lactate', 50820: 'pH_1', 51491: 'pH_2', 50831: 'pH_3', 51094: 'pH_4',
    50818: 'pCO2', 50809: 'Glucose_1', 51478: 'Glucose_2', 50931: 'Glucose_3',
    50902: 'Chloride', 50821: 'pO2', 50833: 'Potassium_1', 50971: 'Potassium_2',
    50862: 'Albumin', 50882: 'Bicarbonate', 50885: 'Bilirubin_Total',
    50802: 'Base_Excess', 50983: 'Sodium', 50868: 'Anion_Gap', 50825: 'Temperature',
    51222: 'Hemoglobin_1', 50811: 'Hemoglobin_2', 51221: 'Hematocrit_1', 51480: 'Hematocrit_2',
    220045: 'Heart_Rate_1', 211: 'Heart_Rate_2', 220210: 'Respiratory_Rate_1', 618: 'Respiratory_Rate_2'
}

# Rename columns using the mapping
labevents_pivot.rename(columns=itemid_to_name, inplace=True)
labevents_pivot.head()

C:\Users\kashann\AppData\Local\Temp\ipykernel_22136\3387768664.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  labevents_df = pd.read_sql_query(labevents_query, con)


itemid,subject_id,Base_Excess,Glucose_1,Hemoglobin_2,Lactate,pCO2,pH_1,pO2,Albumin,Anion_Gap,Bicarbonate,Bilirubin_Total,Chloride,Glucose_3,Potassium_2,Sodium,Hematocrit_1,Hemoglobin_1,pH_2
0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,3,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,4,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,6,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0


In [67]:
print(labevents_pivot.isna().sum())

itemid
subject_id         0
Base_Excess        0
Glucose_1          0
Hemoglobin_2       0
Lactate            0
pCO2               0
pH_1               0
pO2                0
Albumin            0
Anion_Gap          0
Bicarbonate        0
Bilirubin_Total    0
Chloride           0
Glucose_3          0
Potassium_2        0
Sodium             0
Hematocrit_1       0
Hemoglobin_1       0
pH_2               0
dtype: int64


In [68]:
noteevents_query = query_schema+ """
SELECT subject_id, hadm_id, text, category AS note_category
FROM noteevents
"""
noteevents_df = pd.read_sql_query(noteevents_query, con)
noteevents_df['text'] = noteevents_df['text'].str.replace('\r\n', ' ', regex=False)
noteevents_df['text'] = noteevents_df['text'].str.replace('\n', ' ', regex=False)
noteevents_df['text'] = noteevents_df['text'].str.replace('\r', ' ', regex=False)
# 1. Remove commas in text
noteevents_df['text'] = noteevents_df['text'].str.replace(',', ' ', regex=False)

# 2. Handle quotes within text
noteevents_df['text'] = noteevents_df['text'].str.replace('"', '""')



# 4. Remove leading/trailing whitespace
noteevents_df['text'] = noteevents_df['text'].str.strip()

# 5. Remove control characters
noteevents_df['text'] = noteevents_df['text'].str.replace(r'[\x00-\x1F\x7F]', '', regex=True)

# 6. Remove or replace special Unicode characters (if necessary)
noteevents_df['text'] = noteevents_df['text'].apply(lambda x: ''.join(c for c in str(x) if ord(c) < 128))

# 7. Handle missing values
noteevents_df['text'].fillna('NA', inplace=True)

# 8. Remove tabs or special delimiters
noteevents_df['text'] = noteevents_df['text'].str.replace('\t', ' ', regex=False)


C:\Users\kashann\AppData\Local\Temp\ipykernel_22136\487572764.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  noteevents_df = pd.read_sql_query(noteevents_query, con)
C:\Users\kashann\AppData\Local\Temp\ipykernel_22136\487572764.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  noteevents_df['text'].fillna('NA', inplace=True)


In [69]:
 #merge with noteevents
data_before_merge = data.copy()
df = data.merge(noteevents_df, on=['subject_id', 'hadm_id'], how='left')
df_pivot = df.pivot_table(index='subject_id', columns='note_category', values='text', aggfunc='first').reset_index()
df_final = data_before_merge.merge(df_pivot, on='subject_id', how='left')
print(df_final.isna().sum())

subject_id                  0
hadm_id                     0
icustay_id                  0
dbsource                    0
los                         3
ethnicity                   0
diagnosis                   1
gender                      0
age                         0
mortality_inhospital        0
dischtime                   0
Case Management         41196
Consult                 42208
Discharge summary        1189
ECG                      4927
Echo                    19639
General                 38213
Nursing                 32233
Nursing/other           17069
Nutrition               38538
Pharmacy                42159
Physician               32314
Radiology                6317
Rehab Services          39828
Respiratory             37654
Social Work             40516
dtype: int64


In [70]:
df_final= df_final.dropna(subset=['los'])
df_final= df_final.dropna(subset=['diagnosis'])
print(df_final.isna().sum())

subject_id                  0
hadm_id                     0
icustay_id                  0
dbsource                    0
los                         0
ethnicity                   0
diagnosis                   0
gender                      0
age                         0
mortality_inhospital        0
dischtime                   0
Case Management         41192
Consult                 42204
Discharge summary        1187
ECG                      4927
Echo                    19638
General                 38209
Nursing                 32229
Nursing/other           17069
Nutrition               38534
Pharmacy                42155
Physician               32310
Radiology                6317
Rehab Services          39824
Respiratory             37650
Social Work             40512
dtype: int64


In [72]:
#merge with labevents
new_data= df_final.merge(labevents_pivot, on='subject_id', how='inner')
print(new_data.isna().sum())

subject_id                  0
hadm_id                     0
icustay_id                  0
dbsource                    0
los                         0
ethnicity                   0
diagnosis                   0
gender                      0
age                         0
mortality_inhospital        0
dischtime                   0
Case Management         41061
Consult                 42073
Discharge summary        1137
ECG                      4852
Echo                    19522
General                 38082
Nursing                 32117
Nursing/other           16989
Nutrition               38403
Pharmacy                42024
Physician               32196
Radiology                6251
Rehab Services          39694
Respiratory             37520
Social Work             40386
Base_Excess                 0
Glucose_1                   0
Hemoglobin_2                0
Lactate                     0
pCO2                        0
pH_1                        0
pO2                         0
Albumin   

In [73]:

cols = new_data.columns.tolist()
cols.remove('mortality_inhospital')
cols.append('mortality_inhospital')
new_data = new_data[cols]
new_data.head()

,subject_id,hadm_id,icustay_id,dbsource,los,ethnicity,diagnosis,gender,age,dischtime,...,Bicarbonate,Bilirubin_Total,Chloride,Glucose_3,Potassium_2,Sodium,Hematocrit_1,Hemoglobin_1,pH_2,mortality_inhospital
0,268,110404,280836,carevue,3.2490,HISPANIC OR LATINO,DYSPNEA,F,66.030075,18/02/2198 3:55,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1
1,269,106296,206613,carevue,3.2788,WHITE,SEPSIS;PILONIDAL ABSCESS,M,40.127294,27/11/2170 18:00,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0
2,270,188028,220345,carevue,2.8939,UNKNOWN/NOT SPECIFIED,CAROTID STENOSIS\CAROTID ANGIOGRAM AND STENT,M,80.133229,27/06/2128 12:31,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0
3,272,164716,210407,carevue,1.6202,WHITE,PULMONARY EMBOLIS,M,67.142139,02/01/2187 14:57,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0
4,273,158689,241507,carevue,1.4862,BLACK/AFRICAN AMERICAN,POLYSUBSTANCE OVERDOSE,M,33.715776,20/04/2141 17:00,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0


In [75]:
#drop dbsource column
new_data.drop('dbsource', axis=1, inplace=True)
new_data['gender'] = new_data['gender'].replace({'M': 0, 'F': 1})
#remove rows with missing values in gender
# new_data= new_data.dropna(subset=['gender'])

KeyError: "['dbsource'] not found in axis"

In [76]:
new_data['ethnicity'].value_counts()

ethnicity
WHITE                                                       29804
BLACK/AFRICAN AMERICAN                                       3802
UNKNOWN/NOT SPECIFIED                                        3355
HISPANIC OR LATINO                                           1126
OTHER                                                         943
UNABLE TO OBTAIN                                              627
ASIAN                                                         625
PATIENT DECLINED TO ANSWER                                    387
ASIAN - CHINESE                                               201
HISPANIC/LATINO - PUERTO RICAN                                193
BLACK/CAPE VERDEAN                                            151
WHITE - RUSSIAN                                               141
BLACK/HAITIAN                                                  87
MULTI RACE ETHNICITY                                           83
WHITE - OTHER EUROPEAN                                         65


In [77]:
# Define the mapping for broader categories, with "Unknown" categories mapped to "Other"
category_mapping = {
    'WHITE - RUSSIAN' : 'White',
    'WHITE - OTHER EUROPEAN': 'White',
    'WHITE - EASTERN EUROPEAN': 'White',
    'WHITE - BRAZILIAN': 'White',

    'WHITE': 'White',
    'BLACK/AFRICAN AMERICAN': 'Black',
    'BLACK/CAPE VERDEAN': 'Black',
    'BLACK/HAITIAN': 'Black',
    'BLACK/AFRICAN': 'Black',
    'HISPANIC OR LATINO': 'Latino',
    'HISPANIC/LATINO - PUERTO RICAN': 'Latino',
    'HISPANIC/LATINO - DOMINICAN': 'Latino',
    'HISPANIC/LATINO - GUATEMALAN': 'Latino',
    'HISPANIC/LATINO - CUBAN': 'Latino',
    'HISPANIC/LATINO - SALVADORAN': 'Latino',
    'HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)': 'Latino',
    'HISPANIC/LATINO - MEXICAN': 'Latino',
    'HISPANIC/LATINO - COLOMBIAN': 'Latino',
    'HISPANIC/LATINO - HONDURAN': 'Latino',
    'HISPANIC/LATINO - OTHER': 'Latino',
    'ASIAN': 'Asian',
    'ASIAN - CHINESE': 'Asian',
    'ASIAN - ASIAN INDIAN': 'Asian',
    'ASIAN - VIETNAMESE': 'Asian',
    'ASIAN - FILIPINO': 'Asian',
    'ASIAN - CAMBODIAN': 'Asian',
    'ASIAN - OTHER': 'Asian',
    'ASIAN - KOREAN': 'Asian',
    'ASIAN - JAPANESE': 'Asian',
    'ASIAN - THAI': 'Asian',
    'OTHER': 'Other',
    'PORTUGUESE': 'Other',
    'CARIBBEAN ISLAND': 'Other',
    'SOUTH AMERICAN': 'Other',
    'MIDDLE EASTERN': 'Other',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Other',
    'AMERICAN INDIAN/ALASKA NATIVE': 'Other',
    'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE': 'Other',
    'MULTI RACE ETHNICITY': 'Other',
    'UNKNOWN/NOT SPECIFIED': 'Other',
    'UNABLE TO OBTAIN': 'Other',
    'PATIENT DECLINED TO ANSWER': 'Other'
}

# Apply the mapping to the 'category' column
new_data['ethnicity'] = new_data['ethnicity'].replace(category_mapping)


In [78]:
new_data['dischtime'] = pd.to_datetime(new_data['dischtime'], format='%d/%m/%Y %H:%M')

df_sorted = new_data.sort_values(by=['subject_id', 'dischtime'], ascending=[True, False])

df_latest_dischtime = df_sorted.groupby('subject_id').first().reset_index()
df_latest_dischtime.drop('dischtime', axis=1, inplace=True)

df_latest_dischtime.head()

,subject_id,hadm_id,icustay_id,los,ethnicity,diagnosis,gender,age,Case Management,Consult,...,Bicarbonate,Bilirubin_Total,Chloride,Glucose_3,Potassium_2,Sodium,Hematocrit_1,Hemoglobin_1,pH_2,mortality_inhospital
0,3,145834,211552,6.0646,White,HYPOTENSION,0,76.577531,None,None,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0
1,4,185777,294638,1.6785,White,"FEVER,DEHYDRATION,FAILURE TO THRIVE",1,47.876768,None,None,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0
2,6,107064,228232,3.6729,White,CHRONIC RENAL FAILURE/SDA,1,65.986018,None,None,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0
3,9,150750,220597,5.3231,Other,HEMORRHAGIC CVA,0,41.817936,None,None,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1
4,11,194540,229441,1.5844,White,BRAIN MASS,1,50.181544,None,None,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0


In [79]:
#create columns for different ethnicity
df_one_hot = pd.get_dummies(df_latest_dischtime['ethnicity'], prefix='ethnicity')

# Step 6: Concatenate the one-hot columns back to the original DataFrame
df = pd.concat([df_latest_dischtime, df_one_hot], axis=1)

In [80]:
new_data.drop('ethnicity', axis=1, inplace=True)

In [81]:
new_data['dischtime'] = pd.to_datetime(new_data['dischtime'], format='%d/%m/%Y %H:%M')

new_data = new_data.sort_values(by=['subject_id', 'dischtime'], ascending=[True, False])

new_data = new_data.groupby('subject_id').first().reset_index()
new_data.drop('dischtime', axis=1, inplace=True)
print(new_data.isna().sum())

subject_id                  0
hadm_id                     0
icustay_id                  0
los                         0
diagnosis                   0
gender                      0
age                         0
Case Management         33199
Consult                 33630
Discharge summary        1112
ECG                      4715
Echo                    17475
General                 31335
Nursing                 27019
Nursing/other           14406
Nutrition               31448
Pharmacy                33612
Physician               27086
Radiology                4959
Rehab Services          32043
Respiratory             30760
Social Work             32636
Base_Excess                 0
Glucose_1                   0
Hemoglobin_2                0
Lactate                     0
pCO2                        0
pH_1                        0
pO2                         0
Albumin                     0
Anion_Gap                   0
Bicarbonate                 0
Bilirubin_Total             0
Chloride  

In [82]:
#for all value_count less than  100 replace with 'Other'
new_data['diagnosis'] = new_data['diagnosis'].mask(new_data['diagnosis'].map(new_data['diagnosis'].value_counts()) < 100, 'Other')

In [83]:
conditions_with_group_names = {
    "PNEUMONIA": "Cardiovascular Conditions",
    "CORONARY ARTERY DISEASE": "Cardiovascular Conditions",
    "CORONARY ARTERY DISEASE\\CORONARY ARTERY BYPASS GRAFT /SDA": "Cardiovascular Conditions",
    "CORONARY ARTERY DISEASE\\CORONARY ARTERY BYPASS GRAFT/SDA": "Cardiovascular Conditions",
    "CORONARY ARTERY DISEASE\\CATH": "Cardiovascular Conditions",
    "ACUTE CORONARY SYNDROME": "Cardiovascular Conditions",
    "MYOCARDIAL INFARCTION": "Cardiovascular Conditions",
    "AORTIC STENOSIS\\AORTIC VALVE REPLACEMENT /SDA": "Cardiovascular Conditions",
    "CHEST PAIN": "Cardiovascular Conditions",
    "CHEST PAIN\\CATH": "Cardiovascular Conditions",
    "CARDIAC ARREST": "Cardiovascular Conditions",

    "INTRACRANIAL HEMORRHAGE": "Neurological Conditions",
    "SUBARACHNOID HEMORRHAGE": "Neurological Conditions",
    "STROKE;TELEMETRY;TRANSIENT ISCHEMIC ATTACK": "Neurological Conditions",
    "SEIZURE": "Neurological Conditions",
    "ALTERED MENTAL STATUS": "Neurological Conditions",
    "SUBDURAL HEMATOMA": "Neurological Conditions",
    "ACUTE SUBDURAL HEMATOMA": "Neurological Conditions",
    "HEAD BLEED": "Neurological Conditions",

    "RESPIRATORY FAILURE": "Respiratory Conditions",
    "DYSPNEA": "Respiratory Conditions",
    "SHORTNESS OF BREATH": "Respiratory Conditions",

    "S/P FALL": "Trauma and Injury",
    "S/P MOTOR VEHICLE ACCIDENT": "Trauma and Injury",
    "BLUNT TRAUMA": "Trauma and Injury",

    "GASTROINTESTINAL BLEED": "Gastrointestinal Conditions",
    "UPPER GI BLEED": "Gastrointestinal Conditions",
    "LOWER GI BLEED": "Gastrointestinal Conditions",
    "PANCREATITIS": "Gastrointestinal Conditions",
    "ACUTE MYOCARDIAL INFARCTION": "Gastrointestinal Conditions",  # Appears here as well due to categorization overlap
    "UPPER GASTROINTESTINAL BLEED": "Gastrointestinal Conditions",
    "LOWER GASTROINTESTINAL BLEED": "Gastrointestinal Conditions",
    "ABDOMINAL PAIN": "Gastrointestinal Conditions",

    "SEPSIS": "Infections",
    "FEVER": "Infections",
    "CELLULITIS": "Infections",
    "OVERDOSE": "Infections",

    "DIABETIC KETOACIDOSIS": "Metabolic and Endocrine Conditions",
    "HYPOTENSION": "Metabolic and Endocrine Conditions",
    "LIVER FAILURE": "Metabolic and Endocrine Conditions",

    "ACUTE RENAL FAILURE": "Renal Conditions"
}
# Apply the mapping to the 'category' column
new_data['diagnosis'] = new_data['diagnosis'].replace(conditions_with_group_names)


In [84]:
new_data['diagnosis'].value_counts()

diagnosis
Other                                 22034
Cardiovascular Conditions              3690
Neurological Conditions                2273
Gastrointestinal Conditions            1967
Infections                             1280
Trauma and Injury                       754
Metabolic and Endocrine Conditions      613
CONGESTIVE HEART FAILURE                486
Respiratory Conditions                  399
Renal Conditions                        167
Name: count, dtype: int64

In [85]:

df_one_hot = pd.get_dummies(new_data['diagnosis'], prefix='diagnosis')


df = pd.concat([new_data, df_one_hot], axis=1)
df.head()

,subject_id,hadm_id,icustay_id,los,diagnosis,gender,age,Case Management,Consult,Discharge summary,...,diagnosis_CONGESTIVE HEART FAILURE,diagnosis_Cardiovascular Conditions,diagnosis_Gastrointestinal Conditions,diagnosis_Infections,diagnosis_Metabolic and Endocrine Conditions,diagnosis_Neurological Conditions,diagnosis_Other,diagnosis_Renal Conditions,diagnosis_Respiratory Conditions,diagnosis_Trauma and Injury
0,3,145834,211552,6.0646,Metabolic and Endocrine Conditions,0,76.577531,None,None,Admission Date: [**2101-10-20**] Discharg...,...,False,False,False,False,True,False,False,False,False,False
1,4,185777,294638,1.6785,Other,1,47.876768,None,None,Admission Date: [**2191-3-16**] Discharge...,...,False,False,False,False,False,False,True,False,False,False
2,6,107064,228232,3.6729,Other,1,65.986018,None,None,Admission Date: [**2175-5-30**] Dischar...,...,False,False,False,False,False,False,True,False,False,False
3,9,150750,220597,5.3231,Other,0,41.817936,None,None,Admission Date: [**2149-11-9**] Dischar...,...,False,False,False,False,False,False,True,False,False,False
4,11,194540,229441,1.5844,Other,1,50.181544,None,None,Admission Date: [**2178-4-16**] ...,...,False,False,False,False,False,False,True,False,False,False


In [86]:
#drop diagnosis column
df.drop('diagnosis', axis=1, inplace=True)
print(df.isna().sum())

subject_id                                          0
hadm_id                                             0
icustay_id                                          0
los                                                 0
gender                                              0
age                                                 0
Case Management                                 33199
Consult                                         33630
Discharge summary                                1112
ECG                                              4715
Echo                                            17475
General                                         31335
Nursing                                         27019
Nursing/other                                   14406
Nutrition                                       31448
Pharmacy                                        33612
Physician                                       27086
Radiology                                        4959
Rehab Services              

In [87]:
# replace True and false with 1 and 0 in all columns
df = df.replace({True: 1, False: 0})


C:\Users\kashann\AppData\Local\Temp\ipykernel_22136\2755497102.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({True: 1, False: 0})


In [88]:
#remove the nursing column
df.drop('Nursing', axis=1, inplace=True)

In [89]:
df['Social Work'].value_counts()
#check how many unique values in the column
df['Social Work'].nunique()

1027

In [90]:
df.head()
cols = df.columns.tolist()


In [91]:
# Function to get unique types for each column
def get_column_types(df):
    column_types = {}
    for col in df.columns:
        types_in_column = df[col].apply(type).unique()
        column_types[col] = types_in_column
    return column_types

# Get the types for each column
column_types = get_column_types(df)

# Display the results
for col, types in column_types.items():
    print(f"Column '{col}' has the following unique types: {types}")

Column 'subject_id' has the following unique types: [<class 'int'>]
Column 'hadm_id' has the following unique types: [<class 'int'>]
Column 'icustay_id' has the following unique types: [<class 'int'>]
Column 'los' has the following unique types: [<class 'float'>]
Column 'gender' has the following unique types: [<class 'int'>]
Column 'age' has the following unique types: [<class 'float'>]
Column 'Case Management ' has the following unique types: [<class 'NoneType'> <class 'str'>]
Column 'Consult' has the following unique types: [<class 'NoneType'> <class 'str'>]
Column 'Discharge summary' has the following unique types: [<class 'str'> <class 'NoneType'>]
Column 'ECG' has the following unique types: [<class 'str'> <class 'NoneType'>]
Column 'Echo' has the following unique types: [<class 'str'> <class 'NoneType'>]
Column 'General' has the following unique types: [<class 'NoneType'> <class 'str'>]
Column 'Nursing/other' has the following unique types: [<class 'str'> <class 'NoneType'>]
Col

In [47]:
df.head()

,subject_id,hadm_id,icustay_id,los,gender,age,Case Management,Consult,Discharge summary,ECG,...,diagnosis_CONGESTIVE HEART FAILURE,diagnosis_Cardiovascular Conditions,diagnosis_Gastrointestinal Conditions,diagnosis_Infections,diagnosis_Metabolic and Endocrine Conditions,diagnosis_Neurological Conditions,diagnosis_Other,diagnosis_Renal Conditions,diagnosis_Respiratory Conditions,diagnosis_Trauma and Injury
0,3,145834,211552,6.0646,0,76.577531,None,None,Admission Date: [**2101-10-20**] Discharg...,Sinus rhythm P-R interval increased Late R wav...,...,0,0,0,0,1,0,0,0,0,0
1,4,185777,294638,1.6785,1,47.876768,None,None,Admission Date: [**2191-3-16**] Discharge...,Sinus tachycardia. Borderline low limb lead vo...,...,0,0,0,0,0,0,1,0,0,0
2,6,107064,228232,3.6729,1,65.986018,None,None,Admission Date: [**2175-5-30**] Dischar...,Lead V2 absent Probable ectopic atrial rhythm ...,...,0,0,0,0,0,0,1,0,0,0
3,9,150750,220597,5.3231,0,41.817936,None,None,Admission Date: [**2149-11-9**] Dischar...,Sinus rhythm Possible LVH with ST-T changes Si...,...,0,0,0,0,0,0,1,0,0,0
4,11,194540,229441,1.5844,1,50.181544,None,None,Admission Date: [**2178-4-16**] ...,Sinus bradycardia with sinus arrhythmia ST-T w...,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
df = df.drop(columns=['subject_id', 'hadm_id', 'icustay_id','los'])

In [93]:
df.to_json("data_with_text.json", orient="records", lines=True)